In [2]:
import sys
sys.path.append('..')
import torch
import numpy as np
from tqdm import trange
from utils.video import read_video, transform_img
from utils.vqvae import Encoder, CompressorConfig

In [3]:
frames = read_video("../examples/sample_video_ecamera.hevc")
frames = np.array([transform_img(x) for x in frames])
frames = torch.from_numpy(frames).permute(0,3,1,2).to(device='cuda').float()

In [4]:
# load model
config = CompressorConfig()
with torch.device('meta'):
  encoder = Encoder(config)
encoder.load_state_dict_from_url('https://huggingface.co/commaai/commavq-gpt2m/resolve/main/encoder_pytorch_model.bin', assign=True)
encoder = encoder.eval().to(device='cuda')

Downloading: "https://huggingface.co/commaai/commavq-gpt2m/resolve/main/encoder_pytorch_model.bin" to /home/batman/.cache/torch/hub/checkpoints/encoder_pytorch_model.bin
100%|██████████| 113M/113M [00:02<00:00, 51.1MB/s] 


In [5]:
# encoding loop
tokens = []
with torch.no_grad():
  for i in trange(len(frames)):
    encoding_indices = encoder(frames[i][None].float())
    tokens.append(encoding_indices)
tokens = torch.cat(tokens, dim=0)

100%|██████████| 1200/1200 [00:08<00:00, 140.24it/s]


In [6]:
# save the tokens! now head over to the decoding notebook
tokens = tokens.cpu().numpy()
np.save("../examples/tokens.npy", tokens)